ROI Analysis

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from statsmodels.stats.anova import AnovaRM
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests

/home/csimmon2/anaconda3/envs/fmri/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# Helper Functions
def partial_eta_squared(aov):
    """Calculate partial eta-squared for ANOVA results."""
    aov = aov.copy()
    aov['pes'] = aov['F Value'] * aov['Den DF'] / (
        aov['F Value'] * aov['Den DF'] + aov['Den DF']
    )
    return aov

def analyze_roi_data(data, experiment='exp1'):
    """Analyze ROI data with multiple comparison correction."""
    results = {}
    all_tests = []
    
    if experiment == 'exp1':
        conditions = ['object', 'scramble']
        rois = ['LO', 'pIPS']
    else:
        conditions = ['tool', 'nontool']
        rois = ['PFS', 'aIPS', 'LO', 'pIPS']
        
    for roi in rois:
        for hemi in ['left', 'right']:
            roi_data = data[(data['roi'] == roi) & (data['hemi'] == hemi)]
            cond1_data = roi_data[roi_data['cond'] == conditions[0]]['mean_act'].values
            cond2_data = roi_data[roi_data['cond'] == conditions[1]]['mean_act'].values
            
            # Calculate statistics
            t_stat, p_val = ttest_rel(cond1_data, cond2_data)
            mean_diff = np.mean(cond1_data - cond2_data)
            se_diff = np.std(cond1_data - cond2_data) / np.sqrt(len(cond1_data))
            d = mean_diff / np.sqrt((np.std(cond1_data)**2 + np.std(cond2_data)**2) / 2)
            
            # Store results
            all_tests.append({
                'roi': f'{hemi}_{roi}',
                'p_value': p_val,
                't_stat': t_stat,
                'mean_diff': mean_diff,
                'se': se_diff,
                'cohens_d': d,
                'n': len(cond1_data),
                f'mean_{conditions[0]}': np.mean(cond1_data),
                f'mean_{conditions[1]}': np.mean(cond2_data)
            })
    
    # Apply Holm-Bonferroni correction
    p_values = [test['p_value'] for test in all_tests]
    _, p_corrected, _, _ = multipletests(p_values, method='holm')
    
    # Update results with corrected p-values
    for test, p_corr in zip(all_tests, p_corrected):
        results[test['roi']] = test
        results[test['roi']]['p_corrected'] = p_corr
    
    return results

def run_rmanova(data, factors):
    """Run repeated measures ANOVA with effect sizes."""
    aov = AnovaRM(data=data, 
                  depvar='mean_act',
                  subject='sub',
                  within=factors).fit()
    
    aov_table = partial_eta_squared(aov.anova_table)
    return aov_table

In [41]:
# Load Data
# Experiment 1
exp1_results = pd.read_csv("results/selectivity/selectivity_summary.csv")
filtered_exp1 = exp1_results[(exp1_results['group'] == 'control') & 
                            (exp1_results['roi'].isin(['LO', 'pIPS'])) &
                            (exp1_results['sub'] != 'sub-084')]

# Experiment 2
exp2_results = pd.read_csv("/user_data/csimmon2/git_repos/ptoc/results/tools/selectivity/selectivity_summarytoolloc.csv")
filtered_exp2 = exp2_results[exp2_results['sub'].str.contains('spaceloc')]

In [42]:
# Experiment 1 ROI Analysis
# Full ANOVA
exp1_anova = run_rmanova(filtered_exp1, ['roi', 'hemi', 'cond'])
print("Experiment 1 ANOVA Results:")
print(exp1_anova)

# ROI Analysis
exp1_roi_results = analyze_roi_data(filtered_exp1, 'exp1')
print("\nExperiment 1 ROI Results:")
for roi, stats in exp1_roi_results.items():
    print(f"\n{roi}:")
    print(f"t({stats['n']-1}) = {stats['t_stat']:.3f}, p = {stats['p_value']:.4f}, p_corr = {stats['p_corrected']:.4f}")
    print(f"Cohen's d = {stats['cohens_d']:.3f}")
    print(f"Mean difference = {stats['mean_diff']:.3f} ± {stats['se']:.3f}")

Experiment 1 ANOVA Results:
                  F Value  Num DF  Den DF        Pr > F       pes
roi             39.763898     1.0    17.0  7.897722e-06  0.975468
hemi            10.301246     1.0    17.0  5.140748e-03  0.911514
cond            55.387486     1.0    17.0  9.623988e-07  0.982266
roi:hemi         2.803147     1.0    17.0  1.123766e-01  0.737060
roi:cond       108.745714     1.0    17.0  8.346158e-09  0.990888
hemi:cond        2.746805     1.0    17.0  1.157867e-01  0.733106
roi:hemi:cond    0.056390     1.0    17.0  8.151334e-01  0.053380

Experiment 1 ROI Results:

left_LO:
t(17) = 8.141, p = 0.0000, p_corr = 0.0000
Cohen's d = 2.474
Mean difference = 2.526 ± 0.301

right_LO:
t(17) = 8.303, p = 0.0000, p_corr = 0.0000
Cohen's d = 2.689
Mean difference = 2.314 ± 0.271

left_pIPS:
t(17) = 3.815, p = 0.0014, p_corr = 0.0028
Cohen's d = 1.233
Mean difference = 0.851 ± 0.217

right_pIPS:
t(17) = 2.760, p = 0.0134, p_corr = 0.0134
Cohen's d = 0.947
Mean difference = 0.549 ± 0.193

In [43]:
# Experiment 2 ROI Analysis
# Full ANOVA
exp2_anova = run_rmanova(filtered_exp2, ['roi', 'hemi', 'cond'])
print("Experiment 2 ANOVA Results:")
print(exp2_anova)

# Separate ANOVAs for dorsal and ventral pathways
dorsal_data = filtered_exp2[filtered_exp2['roi'].isin(['aIPS', 'pIPS'])]
ventral_data = filtered_exp2[filtered_exp2['roi'].isin(['PFS', 'LO'])]

dorsal_anova = run_rmanova(dorsal_data, ['roi', 'hemi', 'cond'])
ventral_anova = run_rmanova(ventral_data, ['roi', 'hemi', 'cond'])

# ROI Analysis
exp2_roi_results = analyze_roi_data(filtered_exp2, 'exp2')
print("\nExperiment 2 ROI Results:")
for roi, stats in exp2_roi_results.items():
    print(f"\n{roi}:")
    print(f"t({stats['n']-1}) = {stats['t_stat']:.3f}, p = {stats['p_value']:.4f}, p_corr = {stats['p_corrected']:.4f}")
    print(f"Cohen's d = {stats['cohens_d']:.3f}")
    print(f"Mean difference = {stats['mean_diff']:.3f} ± {stats['se']:.3f}")
    
# Dorsal pathway ANOVA
dorsal_data = filtered_exp2[filtered_exp2['roi'].isin(['aIPS', 'pIPS'])]
dorsal_anova = run_rmanova(dorsal_data, ['roi', 'hemi', 'cond'])
print("Dorsal Pathway ANOVA Results:")
print(dorsal_anova)

# Ventral pathway ANOVA
ventral_data = filtered_exp2[filtered_exp2['roi'].isin(['PFS', 'LO'])]
ventral_anova = run_rmanova(ventral_data, ['roi', 'hemi', 'cond'])
print("Ventral Pathway ANOVA Results:")
print(ventral_anova)

Experiment 2 ANOVA Results:
                 F Value  Num DF  Den DF        Pr > F       pes
roi            69.788835     3.0    51.0  4.595074e-18  0.985873
hemi           25.586962     1.0    17.0  9.700100e-05  0.962388
cond            1.697378     1.0    17.0  2.100038e-01  0.629270
roi:hemi        1.617712     3.0    51.0  1.967325e-01  0.617987
roi:cond       23.018004     3.0    51.0  1.468259e-09  0.958365
hemi:cond      15.531839     1.0    17.0  1.053597e-03  0.939511
roi:hemi:cond   1.616753     3.0    51.0  1.969531e-01  0.617847

Experiment 2 ROI Results:

left_PFS:
t(17) = -2.917, p = 0.0096, p_corr = 0.0436
Cohen's d = -0.374
Mean difference = -0.230 ± 0.077

right_PFS:
t(17) = -2.545, p = 0.0209, p_corr = 0.0628
Cohen's d = -0.437
Mean difference = -0.293 ± 0.112

left_aIPS:
t(17) = 5.407, p = 0.0000, p_corr = 0.0004
Cohen's d = 0.844
Mean difference = 0.344 ± 0.062

right_aIPS:
t(17) = 3.430, p = 0.0032, p_corr = 0.0191
Cohen's d = 0.729
Mean difference = 0.225 ± 0.064

FC and PPI Connectivity Analysis

In [11]:
# PPI Analysis || need to alter exp 1 to 18 subjects, currently 19.
import sys
import os
import pandas as pd
import numpy as np
from scipy import stats
import scipy
from statsmodels.stats.multitest import fdrcorrection
import statsmodels.api as sm
from nilearn import image, plotting, input_data, glm
from nilearn.input_data import NiftiMasker
import nibabel as nib
from nilearn.datasets import load_mni152_brain_mask, load_mni152_template
from nilearn.glm.first_level import compute_regressor
import warnings
warnings.filterwarnings('ignore')

# Directories
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
output_dir = f'{curr_dir}/tools'
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'

def calculate_effect_size(data):
    """Calculate Cohen's d"""
    return np.mean(data) / np.std(data) if len(data) > 0 else np.nan

def run_stats_analysis(df, value_col='value'):
    """Run Wilcoxon test and calculate effect size"""
    w_stat, p_val = stats.wilcoxon(df[value_col])
    d = calculate_effect_size(df[value_col])
    return pd.Series({
        'W': w_stat,
        'p': p_val,
        'd': d,
        'mean': np.mean(df[value_col]),
        'sem': stats.sem(df[value_col])
    })

# Load data
exp1_df = pd.read_csv(f"{results_dir}/fc_ppi/matchvlad_seed_target_activations.csv")
tools_df = pd.read_csv(f"{output_dir}/roi_ppi_sphere_summary_tools.csv")
nontools_df = pd.read_csv(f"{output_dir}/roi_ppi_sphere_summary_nontools.csv")

# Add condition labels for Exp2
tools_df['condition'] = 'Tools'
nontools_df['condition'] = 'Nontools'
exp2_df = pd.concat([tools_df, nontools_df])

def analyze_experiment(df, seed_rois, target_rois, exp_num):
    results = []
    hemispheres = ['left', 'right']
    
    for seed in seed_rois:
        for seed_hemi in hemispheres:
            for target in target_rois:
                for target_hemi in hemispheres:
                    if exp_num == 1:
                        subset = df[
                            (df['seed_roi'] == seed) & 
                            (df['seed_hemisphere'] == seed_hemi) &
                            (df['target_roi'] == target) &
                            (df['target_hemisphere'] == target_hemi) &
                            (df['analysis'] == 'ppi')
                        ]
                        if len(subset) > 0:
                            stats = run_stats_analysis(subset, value_col='mean_activation')
                            results.append({
                                'seed': f"{seed}_{seed_hemi}",
                                'target': f"{target}_{target_hemi}",
                                **stats
                            })
                    else:
                        for condition in df['condition'].unique():
                            subset = df[
                                (df['seed_roi'] == seed) & 
                                (df['seed_hemi'] == seed_hemi) &
                                (df['target_roi'] == target) &
                                (df['target_hemi'] == target_hemi) &
                                (df['condition'] == condition)
                            ]
                            if len(subset) > 0:
                                stats = run_stats_analysis(subset)
                                results.append({
                                    'seed': f"{seed}_{seed_hemi}",
                                    'target': f"{target}_{target_hemi}",
                                    'condition': condition,
                                    **stats
                                })
    return pd.DataFrame(results)

# Run analyses
exp1_results = analyze_experiment(exp1_df, ['pIPS'], ['LO'], 1)
exp2_results = analyze_experiment(exp2_df, ['pIPS', 'aIPS'], ['LO'], 2)

# Apply FDR correction
for results_df in [exp1_results, exp2_results]:
    if not results_df.empty and 'p' in results_df.columns:
        p_values = results_df['p'].values
        rejected, fdr_corrected = fdrcorrection(p_values)
        results_df['p_fdr'] = fdr_corrected

# Print and save results
print("\nExperiment 1 Results:")
print(exp1_results.round(3).to_string())
print("\nExperiment 2 Results:")
print(exp2_results.round(3).to_string())

# Save detailed results
exp1_results.to_csv(f"{results_dir}/exp1_ppi_stats.csv", index=False)
exp2_results.to_csv(f"{output_dir}/exp2_ppi_stats.csv", index=False)

# Save summary for manuscript
with open(f"{results_dir}/ppi_stats_summary.txt", 'w') as f:
    f.write("Experiment 1 (Object vs Scramble) PPI Results:\n")
    for _, row in exp1_results.iterrows():
        f.write(f"\n{row['seed']} -> {row['target']}\n")
        f.write(f"W = {row['W']:.0f}, p = {row['p']:.3f} (FDR p = {row['p_fdr']:.3f}), d = {row['d']:.2f}\n")
        f.write(f"Mean ± SEM: {row['mean']:.3f} ± {row['sem']:.3f}\n")
    
    f.write("\n\nExperiment 2 (Tools vs Nontools) PPI Results:\n")
    for _, row in exp2_results.iterrows():
        f.write(f"\n{row['seed']} -> {row['target']} ({row['condition']})\n")
        f.write(f"W = {row['W']:.0f}, p = {row['p']:.3f} (FDR p = {row['p_fdr']:.3f}), d = {row['d']:.2f}\n")
        f.write(f"Mean ± SEM: {row['mean']:.3f} ± {row['sem']:.3f}\n")


Experiment 1 Results:
         seed    target     W      p      d   mean    sem  p_fdr
0   pIPS_left   LO_left   3.0  0.000  1.340  0.035  0.006  0.000
1   pIPS_left  LO_right  12.0  0.000  1.156  0.031  0.006  0.000
2  pIPS_right   LO_left  13.0  0.000  1.137  0.028  0.006  0.000
3  pIPS_right  LO_right  20.0  0.001  1.004  0.023  0.005  0.001

Experiment 2 Results:
          seed    target condition      W      p      d   mean    sem  p_fdr
0    pIPS_left   LO_left     Tools  154.0  0.004  0.557  0.019  0.006  0.022
1    pIPS_left   LO_left  Nontools  253.0  0.214  0.197  0.006  0.005  0.286
2    pIPS_left  LO_right     Tools  166.0  0.008  0.526  0.018  0.006  0.025
3    pIPS_left  LO_right  Nontools  248.0  0.187  0.218  0.007  0.005  0.286
4   pIPS_right   LO_left     Tools  152.0  0.004  0.530  0.017  0.006  0.022
5   pIPS_right   LO_left  Nontools  233.0  0.119  0.256  0.010  0.006  0.211
6   pIPS_right  LO_right     Tools  212.0  0.058  0.376  0.015  0.007  0.115
7   pIPS_righ

In [ ]:
# GCA

In [ ]:
# Network